### Prepare the code and setup database

In [2]:
import os
import json
import pandas as pd
import re
import unicodedata
from langchain_community.graphs import Neo4jGraph
from langchain_neo4j import Neo4jVector
from langchain_ollama import OllamaEmbeddings, ChatOllama
from utils.pdf_utils import extract_pdf_text_by_page, chunk_pages
from langchain_core.messages import SystemMessage, HumanMessage

In [4]:
# Load and read data
data_path = "data_json/test_data.json"

with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

df =  pd.read_json(data_path)

In [5]:
# Log-in and setup database connection
url = "bolt://localhost:7687"
username = "neo4j"
password = "password"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

C:\Users\HLER\AppData\Local\Temp\ipykernel_24700\2264422588.py:6: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the `langchain-neo4j package and should be used instead. To use it run `pip install -U `langchain-neo4j` and import as `from `langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


### Define helpful prompts and linkages

In [6]:
entity_types = {
    "document": "Represents a digital file stored in the system, such as a PDF, drawing, or report. Each document node stores metadata like name, path, size, imported date, and template flag.",
    "user": "Represents the person or account who imported, created, or owns the document. Typically identified by username.",
    "file_type": "Represents the document format, such as 'PDF', 'DOCX', or 'DWG'. Shared across multiple documents to avoid duplication.",
    "status": "Represents the current or historical status of a document (e.g., draft, approved, archived). May also hold status codes or timestamps.",
    "file_date": "Represents temporal information related to the document, such as the creation or modification date and time. Useful for time-based queries.",
    "description": "Represents a semantic classification or content type of the document — for example 'PROCEDURE', 'MANUAL', 'DRAWING', 'SPECIFICATION'."
}

relation_types = {
    "IMPORTED_BY": "Indicates which user imported the document into the system.",
    "HAS_FORMAT": "Links a document to its file format (PDF, DOCX, etc.).",
    "HAS_STATUS": "Associates a document with its workflow or approval status.",
    "HAS_FILETIME": "Links a document to its file date and time metadata.",
    "IS_TYPE": "Classifies the document by description or semantic category (e.g., PROCEDURE, MANUAL)."
}

entity_relationship_match = {
    "user": "IMPORTED_BY",
    "file_type": "HAS_FORMAT",
    "status": "HAS_STATUS",
    "file_date": "HAS_FILETIME",
    "description": "IS_TYPE"
}

In [7]:
system_prompt = f"""
You are an intelligent retrieval planner designed to extract **structured query intent** from a user's natural language question,
so that it can be executed against a **Neo4j knowledge graph** of documents and their metadata.

---

### Graph Schema Overview

Each document node (`Document`) is linked to several **entity types** representing its metadata and attributes:
{json.dumps(entity_types, indent=4)}

Each connection between a `Document` node and an entity uses one of the following **relationship types**:
{json.dumps(relation_types, indent=4)}

These relationships describe how documents connect to users, formats, statuses, and time of import.

---

### Your Role

1. **Interpret the user's intent.**
   - Understand what the user wants to *find* (e.g., a document, a status, a date, a user, etc.).
   - Identify *which parts of the graph* (entity types) are relevant.

2. **Extract query parameters.**
   - Map each piece of meaningful information from the user’s question to the **correct entity type** key.
   - Include only information that exists as nodes or relationships in the graph schema.
   - If the user asks about a *specific document*, include `"Document": "<document_name>"`.
   - If they ask about *a property or relation* of that document (like status or date), include both the document and the requested entity type.

3. **Focus on semantics, not surface keywords.**
   - Recognize synonyms, natural expressions, and implied attributes.
   - Example: “Who imported…” → `User`
   - Example: “When was it created?” → `FileDate`
   - Example: “What’s its current state?” → `Status`
   - Example: “Show me all procedures” → `Description: "procedure"`

4. **Return a JSON object** with one key per relevant entity type and the corresponding value.
   - Key names **must match exactly** one of the entity types above.
   - If nothing in the question maps to known entity types, return an **empty JSON object**.

---

### Examples

**User input:**
> Find procedure PDF files imported by haaler on November 9th 2024 with status 0.

**Expected output:**
```json
{{
    "Description": "Procedure",
    "FileType": "PDF",
    "User": "haaler",
    "FileDate": "Wed Nov 9 2024",
    "Status": 0
}}
"""

## Create the graph

In [8]:
# --- HELPER ---
def clear_graph(tx):
    tx.run("MATCH (n) DETACH DELETE n;")

def sanitize(text):
    return str(text).replace('"', '').replace("'", "").replace('{','').replace('}', '')

# Loop through each JSON object and add them to the DB
i = 1
for obj in data:
    print(f"{i}. Inserting document #{obj['uniqueid']} ({obj['Description']})")
    i += 1

    # Cypher query with fixed schema (no dynamic labels or relationships)
    query = """
    MERGE (d:Document {id: $id})
    ON CREATE SET
        d.name        = $name,
        d.title       = $title,
        d.path        = $path,
        d.user        = $user,
        d.description = $description,
        d.format      = $file_format,
        d.size        = $size_str,
        d.imported    = $imported,
        d.template    = $template

    MERGE (u:User {name: $user})
    MERGE (ff:FileType {name: $file_format})
    MERGE (s:Status {code: $status_code})
    MERGE (fd:FileDate {date_str: $file_date, time_str: $file_time})
    MERGE (t:Description {type: $description})

    MERGE (d)-[:IMPORTED_BY]->(u)
    MERGE (d)-[:HAS_FORMAT]->(ff)
    MERGE (d)-[hs:HAS_STATUS]->(s)
      ON CREATE SET hs.at = $status_date
    MERGE (d)-[:HAS_FILETIME]->(fd)
    MERGE (d)-[:IS_TYPE]->(t)
    """

    # Parameters for this record
    params = {
        "id": obj["uniqueid"],
        "name": obj.get("filename", ""),
        "title": obj.get("orig.filename", ""),
        "path": obj.get("path", ""),
        "user": obj.get("User", ""),
        "description": obj.get("Description", ""),
        "file_format": obj.get("FileType", ""),
        "size_str": obj.get("FileSize", ""),
        "imported": obj.get("Imported", ""),
        "template": obj.get("Template", 0),
        "status_code": obj.get("Status", 0),
        "status_date": obj.get("StatusDate", ""),
        "file_date": obj.get("FileDate", ""),
        "file_time": obj.get("FileTime", "")
    }

    # Run safely with parameters
    graph.query(query, params=params)

1. Inserting document #1 (PROCEDURE)
2. Inserting document #2 (REPORT)
3. Inserting document #3 (Onshore Risk Assessment)
4. Inserting document #4 (Jobcard)


## RAG: Embedding and vector search

#### Embedding the data

In [ ]:
ollama_embedding_model = "qwen3-embedding:0.6b"
def embed_neo4j_nodes(node_label, index_name=None, text_props=None, embedding_model=None):
    """
    Create or update Neo4j vector indexes for nodes of a given label.

    Args:
        node_label (str): The Neo4j node label (e.g. "Document", "User", "FileType").
        index_name (str, optional): Name of the vector index (defaults to the node_label).
        text_props (list, optional): Properties to embed into the vector.
        embedding_model (str, optional): Ollama embedding model (defaults to your global one).
    """

    # Default embedding model if none is passed
    if embedding_model is None:
        embedding_model = "embeddinggemma:300m"

    if index_name is None:
        index_name = node_label.lower() + "_index"

    # Default text properties per node type
    if text_props is None:
        match node_label:
            case "Document":
                text_props = ["name", "title", "description", "path", "format"]
            case "User":
                text_props = ["name"]
            case "FileType":
                text_props = ["name"]
            case "Status":
                text_props = ["code"]
            case "FileDate":
                text_props = ["date_str", "time_str"]
            case "Description":
                text_props = ["type"]
            case _:
                text_props = ["name"]  # fallback

    print(f"🔹 Creating/Updating embedding index for '{node_label}' using properties: {text_props}")

    try:
        vector_index = Neo4jVector.from_existing_graph(
            OllamaEmbeddings(model=embedding_model),
            url=url,
            username=username,
            password=password,
            index_name=index_name,
            node_label=node_label,
            text_node_properties=text_props,
            embedding_node_property="embedding",
        )
        print(f"✅ Successfully embedded '{node_label}' nodes into vector index '{index_name}'")
        return vector_index
    except Exception as e:
        print(f"⚠️ Failed to embed '{node_label}': {e}")
        return None


# --- 1️⃣ Embed your main Document nodes ---
embed_neo4j_nodes("Document", index_name="documents", embedding_model=ollama_embedding_model)

# --- 2️⃣ Embed all related entities (Users, FileTypes, etc.) ---
related_labels = ["User", "FileType", "Status", "FileDate", "Description"]

for label in related_labels:
    embed_neo4j_nodes(label, embedding_model=ollama_embedding_model)


🔹 Creating/Updating embedding index for 'Document' using properties: ['name', 'title', 'description', 'path', 'format']
✅ Successfully embedded 'Document' nodes into vector index 'documents'
🔹 Creating/Updating embedding index for 'User' using properties: ['name']
✅ Successfully embedded 'User' nodes into vector index 'user_index'
🔹 Creating/Updating embedding index for 'FileType' using properties: ['name']
✅ Successfully embedded 'FileType' nodes into vector index 'filetype_index'
🔹 Creating/Updating embedding index for 'Status' using properties: ['code']
✅ Successfully embedded 'Status' nodes into vector index 'status_index'
🔹 Creating/Updating embedding index for 'FileDate' using properties: ['date_str', 'time_str']
✅ Successfully embedded 'FileDate' nodes into vector index 'filedate_index'
🔹 Creating/Updating embedding index for 'Description' using properties: ['type']
✅ Successfully embedded 'Description' nodes into vector index 'description_index'


#### Embedding the user query

In [75]:
embeddings_client = OllamaEmbeddings(model=ollama_embedding_model)

def create_embedding(text):
    return embeddings_client.embed_query(text)

In [76]:
def create_query(text, threshold=0.65):
    query_data = json.loads(text)
    embeddings_data = []
    match_data = []
    similarity_data = []

    # Build WITH clause
    for key in query_data:
        if key.lower() != "document":
            embeddings_data.append(f"${key}Embedding AS {key}Embedding")

    query = "WITH " + ",\n".join(embeddings_data) + "\nMATCH (d:Document)\n"

    # Dynamically build MATCH and WHERE clauses only for keys present
    for key in query_data:
        if key.lower() != "document":
            relationship = entity_relationship_match.get(key.lower())
            if relationship:
                var_name = f"{key}Var"
                match_data.append(f"MATCH (d)-[:{relationship}]->({var_name}:{key})")
                similarity_data.append(
                    f"gds.similarity.cosine({var_name}.embedding, ${key}Embedding) > {threshold}"
                )

    # Add matches only if they exist
    if match_data:
        query += "\n".join(match_data) + "\n"

    # Add where clause safely
    if similarity_data:
        query += "WHERE " + " AND ".join(similarity_data) + "\n"

    query += "RETURN d"
    return query



In [77]:
def query_graph(response):
    embeddingsParams = {}
    query = create_query(response)
    query_data = json.loads(response)
    for key, val in query_data.items():
        embeddingsParams[f"{key}Embedding"] = create_embedding(val)
    result = graph.query(query, params=embeddingsParams)
    return result

In [78]:
example_response = '''{
    "Description": "Procedure",
    "FileType": "PDF",
    "User": "haaler",
    "FileDate": "Wed Nov  5 2025",
    "Status": "0"
}'''

result = query_graph(example_response)

#### Vector Search Result: Similarity Search

In [79]:
# --- Result display ---
if not result or len(result) == 0:
    print("⚠️ No matching documents found.\n")
else:
    print(f"✅ Found {len(result)} matching document(s):\n")

    for record in result:
        # Each record has a key 'd' from "RETURN d"
        doc = record.get("d", {})

        # Safely extract properties
        doc_id = doc.get("id", "N/A")
        doc_name = doc.get("name", "Unnamed Document")
        doc_title = doc.get("title", "")
        doc_type = doc.get("format", doc.get("type", "Unknown Format"))
        doc_user = doc.get("user", "Unknown User")
        doc_date = doc.get("imported", "")

        # Pretty print summary
        print(f"🗎  {doc_name} ({doc_id})")
        print(f"   ├ Title: {doc_title}")
        print(f"   ├ Type:  {doc_type}")
        print(f"   ├ User:  {doc_user}")
        print(f"   └ Imported: {doc_date}\n")


✅ Found 1 matching document(s):

🗎  45321-J-KA-0001 (1)
   ├ Title: Kasra Taheri.pdf
   ├ Type:  PDF
   ├ User:  haaler
   └ Imported: Wed Nov  5 13:29:07 2025 CET



## GraphRAG: Graph Traversal

In [65]:
# Adjust relationships_threshold to return documents that share more or fewer connected entities
def query_similar_documents(document_id, relationships_threshold=3):
    similar_docs = []

    # --- 1️⃣ Documents with the same Description type (e.g., PROCEDURE, MANUAL, DRAWING) ---
    query_same_description = """
        MATCH (d:Document {id: $document_id})-[:IS_TYPE]->(desc:Description)
        MATCH (d)-->(entity)
        WHERE NOT entity:Description
        MATCH (n:Document)-[:IS_TYPE]->(desc)
        MATCH (n)-->(commonEntity)
        WHERE commonEntity = entity AND d.id <> n.id
        RETURN DISTINCT n;
    """

    result_description = graph.query(query_same_description, params={"document_id": int(document_id)})

    # --- 2️⃣ Documents sharing at least N other entities (e.g., same User, FileType, or Status) ---
    query_common_entities = """
        MATCH (d:Document {id: $document_id})-->(entity),
              (n:Document)-->(entity)
        WHERE d.id <> n.id
        WITH n, COUNT(DISTINCT entity) AS commonEntities
        WHERE commonEntities >= $threshold
        RETURN n;
    """

    result_common_entities = graph.query(
        query_common_entities,
        params={"document_id": int(document_id), "threshold": relationships_threshold}
    )

    # --- 3️⃣ Combine results and deduplicate ---
    for r in result_description:
        similar_docs.append({
            "id": r['n'].get('id', 'N/A'),
            "name": r['n'].get('name', 'Unnamed Document')
        })

    for r in result_common_entities:
        result_id = r['n'].get('id', 'N/A')
        if not any(item['id'] == result_id for item in similar_docs):
            similar_docs.append({
                "id": result_id,
                "name": r['n'].get('name', 'Unnamed Document')
            })

    # --- 4️⃣ Return results ---
    return similar_docs


In [74]:
document_ids = [1, 2, 3]  # example IDs

for document_id in document_ids:
    print(f"📄 Similar documents for document #{document_id}:\n")
    result = query_similar_documents(document_id)
    
    if not result:
        print("   ⚠️ No similar documents found.\n")
        continue

    for r in result:
        print(f"   • {r['name']} (ID: {r['id']})")
    print("\n")


📄 Similar documents for document #1:

   ⚠️ No similar documents found.

📄 Similar documents for document #2:

   • 83743-P-OR-0002 (ID: 3)


📄 Similar documents for document #3:

   • 32352-F-RA-0003 (ID: 2)




In [67]:
def query_db(params):
    matches = []
    # Query the database using your embedding-based GraphRAG query
    result = query_graph(params)

    for r in result:
        doc = r.get("d", {})  # Neo4j returns the variable alias 'd' (from RETURN d)
        doc_id = doc.get("id", "N/A")
        doc_name = doc.get("name", "Unnamed Document")

        matches.append({
            "id": doc_id,
            "name": doc_name
        })
    return matches


In [70]:
def similarity_search(prompt, threshold=0.60):
    """
    Perform semantic similarity search over Document nodes based on the given text prompt.

    Args:
        prompt (str): The natural-language search text (e.g., "procedure PDF imported by haaler").
        threshold (float): Minimum cosine similarity threshold (0–1).

    Returns:
        list[dict]: Matching documents with their IDs and names.
    """
    matches = []

    # Convert query text into an embedding using your Ollama embedding model
    embedding = create_embedding(prompt)

    # Cypher query: compare embedding similarity to all Document nodes
    query = '''
        WITH $embedding AS inputEmbedding
        MATCH (d:Document)
        WHERE d.embedding IS NOT NULL
          AND gds.similarity.cosine(inputEmbedding, d.embedding) > $threshold
        RETURN d
        ORDER BY gds.similarity.cosine(inputEmbedding, d.embedding) DESC
        LIMIT 10
    '''

    result = graph.query(query, params={'embedding': embedding, 'threshold': threshold})

    for r in result:
        d = r.get('d', {})
        doc_id = d.get('id', 'N/A')
        doc_name = d.get('name', 'Unnamed Document')

        matches.append({
            "id": doc_id,
            "name": doc_name
        })

    return matches


In [73]:
prompt_similarity = "Find documents about procedures imported by haaler"
print(similarity_search(prompt_similarity))

[{'id': 1, 'name': '45321-J-KA-0001'}]
